In [1]:
#!pip install sentence_transformers
#!pip install emoji
#!pip install datasets
#!pip install imblearn

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.0/226.0 KB 316.2 kB/s eta 0:00:00m eta 0:00:010:00:01


In [1]:
import pandas as pd
from sklearn.decomposition import PCA
from sentence_transformers import SentenceTransformer
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from transformers import AutoTokenizer,AutoModelForMaskedLM, AutoModelForSequenceClassification
import time
from sklearn.metrics import f1_score
from sklearn.feature_extraction.text import CountVectorizer
import re
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.linear_model import LogisticRegression
from transformers import pipeline
import torch
from imblearn.over_sampling import SMOTE


[nltk_data] Downloading package stopwords to /home/vant/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/vant/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /home/vant/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
from pysentimiento.pysentimiento import create_analyzer
from pysentimiento.pysentimiento.preprocessing import preprocess_tweet

In [2]:
model = SentenceTransformer('espejelomar/sentece-embeddings-BETO')
#tokenizer = AutoTokenizer.from_pretrained('espejelomar/sentece-embeddings-BETO')

In [4]:
analyzer = create_analyzer(task="emotion", lang="es")

In [3]:
train=pd.read_csv("test.csv")

In [6]:
train.head()

,Unnamed: 0,index,tweet,humor
0,0,52830,-Mamá en la escuela me dicen gorda -Pobresilla...,0
1,1,78883,"No te sientas diferente, da igual si eres negr...",1
2,2,78926,Si esta asi.. SUPER SI.. y que se pongan celos...,0
3,3,61844,—Bebé ¿Me veo gorda con este vestido?\n—¡No mi...,0
4,4,78830,Las mujeres solo desean 2 cosas en la vida: co...,0


In [4]:
texts=train['tweet']
#texts=[]

In [81]:
for elem in textos:
    texts.append(preprocess_tweet(elem))

In [87]:
print(texts[0],textos[0])

Mi celular tiene una aplicación que te hace ver todo gordo y feo. ¿En serio? ¿Cómo se llama? Cámara. Mi celular tiene una aplicación que te hace ver todo gordo y feo. —¿En serio? ¿Cómo se llama? —Cámara.


In [8]:
## Sacar las emociones a los tweets y montar el dataframe con emb y emotions.

emotions=[]
for elem in texts:
    e=analyzer.predict(elem)
    
    values=[]
    for k,v in e.probas.items():
        values.append(v)
    emotions.append(values)
    
    

In [9]:
etiquetas=[]
for elem in e.probas.keys():
    etiquetas.append(elem)

In [10]:
emotions[0]

[0.6790232062339783,
 0.10766538977622986,
 0.1620318442583084,
 0.029340621083974838,
 0.01272048894315958,
 0.0056531974114477634,
 0.003565221093595028]

In [90]:
etiquetas

['others', 'joy', 'sadness', 'anger', 'surprise', 'disgust', 'fear']

In [91]:
train['humor'].value_counts()

0    1802
1     869
Name: humor, dtype: int64

In [6]:
emb=[]
cont=0
for e in texts:
    cont+=1
    print(f'Analizando {cont}')

    emb.append(model.encode(e.strip()))

Analizando 1
-Mamá en la escuela me dicen gorda -Pobresilla -Sí, ¿verdad que no estoy gorda? -No, digo, pobre silla, la vas a romper. - MAMAAAAÁ!!!
Analizando 2
No te sientas diferente, da igual si eres negro, amarillo o normal
Analizando 3
Si esta asi.. SUPER SI.. y que se pongan celosas las mamás con sobrepeso
Analizando 4
—Bebé ¿Me veo gorda con este vestido?
—¡No mi amor, te vez gorda con todos!
Analizando 5
Las mujeres solo desean 2 cosas en la vida: comer y perder peso.
Analizando 6
—Mamá, hasta en mis sueños me dicen gorda. 
 —Tranquila, hija, solo son pesadillas. 
—¡¿QUESADILLAS?! ¿Dónde?  XD
Analizando 7
Diferencias entre mi suegra y una foca

Una es gorda, tiene bigote y huele a pescado

La  otra vive en el mar
Analizando 8
Mi vecina es tan gorda que en vez de autoestima, tiene AutoMac
Analizando 9
“Todas las amigas de mi novio son putas, menos la gorda, ella es bien linda”.    -Todas las mujeres.
Analizando 10
STRP   HASHTAG   HASHTAG   HASHTAG   MENTION
En que se diferencia

Analizando 65
Lo géneros son como las torres gemelas, antes eran dos pero ahora es un tema sensible.
Analizando 66
¿Cómo metes a 100 niños de Somalia dentro de un Ford?.
Tirando un pedazo de pan adentro.
Analizando 67
Lo géneros son como las torres gemelas, antes eran dos pero ahora es un tema sensible.
Analizando 68
MENTION Siempre te tocan los locos y los tontos. Estás segura de que por error no te has bajado una app para ligar con subnormales profundos? "Adopta un tontopollas, ellos (aunque negro) también tienen corazón".
Analizando 69
- ¿Qué diferencia a un pizza de un judío?
+ Que la pizza no grita cuando lo metes en el horno

 hum of   HASHTAG   HASHTAG   MENTION
Analizando 70
Las mujeres son como los chinos: nadie las entiende y están dominando el mundo.
Analizando 71
Entre chinos: -Me voy a metel en un lío -¿En un ploblema? -No, en un lio a bañalme.
Analizando 72
Te tiras una chilena y eres heroe nacional. Te tiras una ecuatoriana... y te expulsan de la selección! by  MENTION  

KeyboardInterrupt: 

In [12]:
dataframe=pd.DataFrame(emotions,columns=etiquetas)

columnas=[f"Columna{i}" for i in range(len(emb[0]))]
df=pd.DataFrame(emb,columns=columnas)

In [13]:
dataframe

,others,joy,sadness,anger,surprise,disgust,fear
0,0.679023,0.107665,0.162032,0.029341,0.012720,0.005653,0.003565
1,0.959532,0.009115,0.014948,0.009281,0.000933,0.004011,0.002180
2,0.927571,0.005238,0.021650,0.022811,0.006689,0.009088,0.006953
3,0.537640,0.036640,0.115944,0.234139,0.012696,0.046911,0.016030
4,0.985599,0.007593,0.003549,0.001013,0.000610,0.000549,0.001087
...,...,...,...,...,...,...,...
773,0.039392,0.001972,0.006088,0.808178,0.001724,0.140737,0.001908
774,0.004310,0.001010,0.004892,0.807120,0.002689,0.177845,0.002136
775,0.016737,0.002757,0.005695,0.772355,0.002001,0.198583,0.001872
776,0.934931,0.028980,0.009138,0.008992,0.014133,0.002063,0.001763


In [14]:
df

,Columna0,Columna1,Columna2,Columna3,Columna4,Columna5,Columna6,Columna7,Columna8,Columna9,...,Columna758,Columna759,Columna760,Columna761,Columna762,Columna763,Columna764,Columna765,Columna766,Columna767
0,-0.477419,0.333013,-0.342612,0.536009,-0.550707,-0.168272,-0.730655,0.104565,0.107547,0.177411,...,0.428949,-0.327656,-0.423109,0.572448,-0.084187,0.177116,-0.330961,-0.201172,0.088209,0.799183
1,-0.852175,-0.456203,-0.716483,-0.644707,-0.848353,0.053922,-0.389059,0.180980,0.086490,1.004192,...,0.876312,-1.100981,-0.279185,-0.099568,-0.655890,0.356255,0.008993,-0.107915,-0.589259,0.044471
2,-0.347474,-0.220565,-0.991643,-0.024514,-0.140533,-0.200014,0.113413,0.032648,0.117641,0.530441,...,0.507084,-0.489755,-0.260202,0.695309,0.084397,-0.249644,-0.206611,0.248168,0.327281,0.578134
3,-0.428616,-0.207868,-0.574229,0.347054,-0.373647,-0.258817,-0.473959,-0.256468,-0.126372,0.519432,...,0.637227,-0.690802,-0.503891,0.469311,-0.210276,0.173068,-0.399728,-0.295321,-0.400342,0.963657
4,-0.629425,-0.663396,-0.775956,0.224777,-0.474827,-0.612141,-0.346525,0.858388,-0.420422,-0.327433,...,0.837056,-0.544947,0.231457,0.837975,-0.685083,0.454363,0.891211,-0.537198,0.382588,1.194219
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
773,-1.257880,0.481681,-0.299571,0.605112,0.131609,-0.538369,-0.400374,0.507537,0.371316,-0.134474,...,0.605491,0.119960,0.186015,0.406305,0.376034,-0.240950,-0.200172,-0.090800,0.222161,0.941365
774,-0.869624,0.263838,-0.261625,0.848253,0.134009,-1.041306,-0.045765,0.650488,-0.030841,0.271911,...,0.285433,0.265240,-0.114828,0.220914,0.178620,-0.121222,0.003131,-0.239336,0.347573,0.446492
775,-0.882013,0.377831,-0.138815,0.138060,0.339364,-0.573873,-0.329129,-0.207239,0.392666,-0.373769,...,-0.011498,-0.652681,0.225859,-0.422827,-0.249400,-0.084787,0.576665,-0.022155,0.765587,0.243946
776,-0.716993,-0.282887,-0.509519,1.010377,-0.357664,-0.386435,-0.803936,0.079764,-0.125711,0.090443,...,-0.020254,-0.143367,-0.081957,0.221083,-0.580917,-0.097251,0.868421,-0.358304,-0.650283,0.188109


In [15]:
dtotal=pd.concat([df,dataframe],axis=1)

In [16]:
dtotal

,Columna0,Columna1,Columna2,Columna3,Columna4,Columna5,Columna6,Columna7,Columna8,Columna9,...,Columna765,Columna766,Columna767,others,joy,sadness,anger,surprise,disgust,fear
0,-0.477419,0.333013,-0.342612,0.536009,-0.550707,-0.168272,-0.730655,0.104565,0.107547,0.177411,...,-0.201172,0.088209,0.799183,0.679023,0.107665,0.162032,0.029341,0.012720,0.005653,0.003565
1,-0.852175,-0.456203,-0.716483,-0.644707,-0.848353,0.053922,-0.389059,0.180980,0.086490,1.004192,...,-0.107915,-0.589259,0.044471,0.959532,0.009115,0.014948,0.009281,0.000933,0.004011,0.002180
2,-0.347474,-0.220565,-0.991643,-0.024514,-0.140533,-0.200014,0.113413,0.032648,0.117641,0.530441,...,0.248168,0.327281,0.578134,0.927571,0.005238,0.021650,0.022811,0.006689,0.009088,0.006953
3,-0.428616,-0.207868,-0.574229,0.347054,-0.373647,-0.258817,-0.473959,-0.256468,-0.126372,0.519432,...,-0.295321,-0.400342,0.963657,0.537640,0.036640,0.115944,0.234139,0.012696,0.046911,0.016030
4,-0.629425,-0.663396,-0.775956,0.224777,-0.474827,-0.612141,-0.346525,0.858388,-0.420422,-0.327433,...,-0.537198,0.382588,1.194219,0.985599,0.007593,0.003549,0.001013,0.000610,0.000549,0.001087
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
773,-1.257880,0.481681,-0.299571,0.605112,0.131609,-0.538369,-0.400374,0.507537,0.371316,-0.134474,...,-0.090800,0.222161,0.941365,0.039392,0.001972,0.006088,0.808178,0.001724,0.140737,0.001908
774,-0.869624,0.263838,-0.261625,0.848253,0.134009,-1.041306,-0.045765,0.650488,-0.030841,0.271911,...,-0.239336,0.347573,0.446492,0.004310,0.001010,0.004892,0.807120,0.002689,0.177845,0.002136
775,-0.882013,0.377831,-0.138815,0.138060,0.339364,-0.573873,-0.329129,-0.207239,0.392666,-0.373769,...,-0.022155,0.765587,0.243946,0.016737,0.002757,0.005695,0.772355,0.002001,0.198583,0.001872
776,-0.716993,-0.282887,-0.509519,1.010377,-0.357664,-0.386435,-0.803936,0.079764,-0.125711,0.090443,...,-0.358304,-0.650283,0.188109,0.934931,0.028980,0.009138,0.008992,0.014133,0.002063,0.001763


In [17]:
dtotal.to_csv("test_procesado.csv")

In [74]:
vectorizer=CountVectorizer()
vectorizer.fit(texts)
X_bag=vectorizer.transform(texts)
print(vectorizer.vocabulary_)

{'mi': 6049, 'celular': 1678, 'tiene': 9127, 'una': 9483, 'aplicación': 687, 'que': 7793, 'te': 9005, 'hace': 4473, 'ver': 9658, 'todo': 9184, 'gordo': 4326, 'feo': 3961, 'en': 3325, 'serio': 8560, 'cómo': 2537, 'se': 8443, 'llama': 5474, 'cámara': 2526, 'esta': 3644, 'vida': 9725, 'me': 5935, 'tocó': 9176, 'tener': 9051, 'mala': 5690, 'suerte': 8866, 'espero': 3627, 'la': 5266, 'proxima': 7680, 'toque': 9221, 'ser': 8548, 'flaco': 4019, 'tu': 9402, 'mamá': 5728, 'es': 3515, 'taaan': 8949, 'taan': 8950, 'obesa': 6567, 'cuando': 2421, 'pasa': 6923, 'frente': 4095, 'tv': 9431, 'pierdes': 7239, 'como': 2012, 'comerciales': 1989, 'fb': 3912, 'tía': 9445, 'dijo': 3009, 'tengo': 9059, 'memoria': 5968, 'de': 2591, 'elefante': 3252, 'le': 5333, 'respondí': 8165, 'buenisimo': 1322, 'juego': 5187, 'con': 2084, 'cara': 1542, 'el': 3247, 'cuerpo': 2452, 'colegio': 1945, 'dicen': 2955, 'gorda': 4317, 'ay': 988, 'marrana': 5864, 'no': 6491, 'les': 5369, 'hagas': 4502, 'caso': 1603, 'mariana': 5829, 

In [75]:
X=X_bag.toarray()

In [76]:
len(X[0])

10036

In [80]:
cols=[f"X_bag{i}" for i in range(len(X[0]))]

In [81]:
dbag=pd.DataFrame(X,columns=cols)

In [82]:
dbag

,X_bag0,X_bag1,X_bag2,X_bag3,X_bag4,X_bag5,X_bag6,X_bag7,X_bag8,X_bag9,...,X_bag10026,X_bag10027,X_bag10028,X_bag10029,X_bag10030,X_bag10031,X_bag10032,X_bag10033,X_bag10034,X_bag10035
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2666,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2667,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2668,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2669,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [83]:
dtotal1=pd.concat([dtotal,dbag],axis=1)

In [84]:
dtotal1

,Columna0,Columna1,Columna2,Columna3,Columna4,Columna5,Columna6,Columna7,Columna8,Columna9,...,X_bag10026,X_bag10027,X_bag10028,X_bag10029,X_bag10030,X_bag10031,X_bag10032,X_bag10033,X_bag10034,X_bag10035
0,-0.448494,-0.188356,-0.463675,-0.258665,-0.455674,-0.050351,-0.123209,0.176246,0.284057,0.234793,...,0,0,0,0,0,0,0,0,0,0
1,-1.079639,-0.253492,-0.113903,-0.569727,-0.177083,-0.306786,0.551178,0.124691,-0.291144,0.087227,...,0,0,0,0,0,0,0,0,0,0
2,-0.031088,0.097844,-0.768265,-0.082579,-0.133160,0.101733,0.077331,0.161244,-0.233863,0.384410,...,0,0,0,0,0,0,0,0,0,0
3,-0.574976,-0.166939,0.018835,0.388062,-0.722610,0.068617,0.015301,0.012036,0.086840,0.475127,...,0,0,0,0,0,0,0,0,0,0
4,-0.972600,-0.235676,-0.445274,0.312496,-0.346444,-0.207861,-0.819376,0.335912,0.336409,0.101061,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2666,-0.734115,0.020445,-0.847210,0.843931,-0.131933,-0.378510,-0.034009,0.308299,0.299575,0.025381,...,0,0,0,0,0,0,0,0,0,0
2667,-1.297719,0.106787,-0.773393,0.374609,0.132493,-0.731110,0.173032,0.540405,0.413045,0.176917,...,0,0,0,0,0,0,0,0,0,0
2668,-0.450039,-0.496933,-0.904006,0.536732,-0.160713,-0.036997,-0.139085,0.386301,-0.145777,0.646092,...,0,0,0,0,0,0,0,0,0,0
2669,-0.524159,0.471604,-0.323636,0.580177,-0.523063,0.451996,0.236529,0.582883,-0.461424,0.094865,...,0,0,0,0,0,0,0,0,0,0


## TRAINING

In [85]:
x_train,x_test,y_train,y_test = train_test_split(dtotal1,train['humor'],test_size=0.2,random_state=42)

In [86]:
from sklearn.model_selection import GridSearchCV
start = time.time() 

svm_classifier = GridSearchCV(estimator=SVC(), param_grid={'C':[0.01,0.1,1,10,20], 'kernel': ('linear', 'rbf')})
svm_classifier.fit(x_train,y_train)

end = time.time()
process = round(end-start,2)
print("Support Vector Machine Classifier has fitted, this process took {} seconds".format(process))

Support Vector Machine Classifier has fitted, this process took 2634.89 seconds


In [87]:
svm_classifier.best_params_

{'C': 1, 'kernel': 'rbf'}

In [88]:
svm_classifier.score(x_test,y_test)

0.8205607476635514

In [89]:
ypred=svm_classifier.predict(x_test)

In [90]:
f1_score(y_test,ypred,pos_label=1)

0.6862745098039215